# Required Packages

In [ ]:
import numpy as np
import os
import glob
import cv2 as cv
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
import math
import random
import tensorflow_addons as tfa
from focal_loss import SparseCategoricalFocalLoss

In [ ]:
dir_cur = os.getcwd()

# Importing data

In [ ]:
train_x = np.load(r"path\filename")
train_y = np.load(r"path\filename")


test_x = np.load(r"path\filename")
test_y = np.load(r"path\filename")

valid_x = np.load(r"path\filename")
valid_y = np.load(r"path\filename")

print(f"Train Data")
print(f"train_x data shape = {train_x.shape} \ttrain_x data type = {train_x.dtype}")
print(f"train_y data shape = {train_y.shape} \ttrain_y data type = {train_y.dtype}\n\n")

print(f"Test Data")
print(f"test_x data shape = {test_x.shape} \t\ttest_x data type = {test_x.dtype}")
print(f"test_y data shape = {test_y.shape} \t\ttest_y data type = {test_y.dtype}\n\n")

print(f"Validation Data")
print(f"valid_x data shape = {valid_x.shape} \tvalid_x data type = {valid_x.dtype}")
print(f"valid_y data shape = {valid_y.shape} \tvalid_y data type = {valid_y.dtype}")

# Dataset Visulazitaion

In [ ]:
plt.figure(figsize=[12,12])
plt.subplot(3,2,1)
plt.imshow(train_x[0], cmap='gray',vmin=0, vmax=0.3)
plt.title("grayscale train image")
plt.subplot(3,2,2)
plt.imshow(train_y[0],cmap='jet_r')
plt.title("Segmented train image")
plt.subplot(3,2,3)
plt.imshow(valid_x[0],cmap='gray',vmin=0, vmax=0.3)
plt.title("grayscale valid image")
plt.subplot(3,2,4)
plt.imshow(valid_y[0],cmap='jet_r')
plt.title("Segmented valid image")
plt.subplot(3,2,5)
plt.imshow(test_x[0],cmap='gray',vmin=0, vmax=0.3)
plt.title("grayscale test image")
plt.subplot(3,2,6)
plt.imshow(test_y[0],cmap='jet_r')
plt.title("Segmented test image")
plt.show()

# UResNet Structure

In [ ]:
#NetWork Structure

input = layers.Input(shape=train_x[0, :, :, :].shape)

# B1 - Encode
x = layers.Conv2D(filters=64, kernel_size=(4, 4), strides=1, padding='same')(input)
x = layers.BatchNormalization()(x)
E1LC = layers.Activation('relu')(x)
x = layers.MaxPool2D(pool_size=(2, 2))(E1LC)

# B2 - Encode
x = layers.Conv2D(filters=128, kernel_size=(1, 1), strides=1, padding='same')(x)
E2SC = layers.Activation('relu')(x)
x = layers.Conv2D(filters=128, kernel_size=(4, 4), strides=1, padding='same')(E2SC)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=128, kernel_size=(4, 4), strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
E2LC = layers.Add()([x, E2SC])
x = layers.MaxPool2D(pool_size=(2, 2))(E2LC)

# B3 - Encode
x = layers.Conv2D(filters=256, kernel_size=(1, 1), strides=1, padding='same')(x)
E3SC = layers.Activation('relu')(x)
x = layers.Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same')(E3SC)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
E3LC = layers.Add()([x, E3SC])
x = layers.MaxPool2D(pool_size=(2, 2))(E3LC)

# B4 - Encode
x = layers.Conv2D(filters=512, kernel_size=(1, 1), strides=1, padding='same')(x)
E4SC = layers.Activation('relu')(x)
x = layers.Conv2D(filters=512, kernel_size=(4, 4), strides=1, padding='same')(E4SC)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=512, kernel_size=(4, 4), strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
E4LC = layers.Add()([x, E4SC])
x = layers.MaxPool2D(pool_size=(2, 2))(E4LC)

# Middle
x = layers.Conv2D(filters=1024, kernel_size=(1, 1), strides=1, padding='same')(x)
MidSC = layers.Activation('relu')(x)
x = layers.Conv2D(filters=1024, kernel_size=(4, 4), strides=1, padding='same')(MidSC)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=1024, kernel_size=(4, 4), strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Add()([x, MidSC])
x = layers.Conv2DTranspose(filters=1024, kernel_size=(4, 4), strides=2, padding='same', kernel_regularizer =tf.keras.regularizers.l2(l=0.00001))(x)

# B4S - Decode
x = layers.BatchNormalization()(x)
x = layers.concatenate([x,E4LC], axis=3)
x = layers.Conv2D(filters=512, kernel_size=(1, 1), strides=1, padding='same')(x)
D4SC = layers.Activation('relu')(x)
x = layers.Conv2D(filters=512, kernel_size=(4, 4), strides=1, padding='same')(D4SC)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=512, kernel_size=(4, 4), strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Add()([x, D4SC])
x = layers.Conv2DTranspose(filters=512, kernel_size=(4, 4), strides=2, padding='same')(x)

# B3S - Decode
x = layers.BatchNormalization()(x)
x = layers.concatenate([x,E3LC], axis=3)
x = layers.Conv2D(filters=256, kernel_size=(1, 1), strides=1, padding='same')(x)
D3SC = layers.Activation('relu')(x)
x = layers.Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same')(D3SC)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=256, kernel_size=(4, 4), strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Add()([x, D3SC])
x = layers.Conv2DTranspose(filters=256, kernel_size=(4, 4), strides=2, padding='same')(x)

# B2S - Decode
x = layers.BatchNormalization()(x)
x = layers.concatenate([x,E2LC], axis=3)
x = layers.Conv2D(filters=128, kernel_size=(1, 1), strides=1, padding='same')(x)
D2SC = layers.Activation('relu')(x)
x = layers.Conv2D(filters=128, kernel_size=(4, 4), strides=1, padding='same')(D2SC)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=128, kernel_size=(4, 4), strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Add()([x, D2SC])
x = layers.Conv2DTranspose(filters=128, kernel_size=(4, 4), strides=2, padding='same')(x)

# B1S - Decode
x = layers.BatchNormalization()(x)
x = layers.concatenate([x,E1LC], axis=3)
x = layers.Conv2D(filters=64, kernel_size=(1, 1), strides=1, padding='same')(x)
D1SC = layers.Activation('relu')(x)
x = layers.Conv2D(filters=64, kernel_size=(4, 4), strides=1, padding='same')(D1SC)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=64, kernel_size=(4, 4), strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Add()([x, D1SC])

#outputs
output = layers.Conv2D(3, 1, activation="softmax", padding="same")(x)
autoencoder = keras.models.Model(input, output)
autoencoder.summary()

# Required Metrics

In [ ]:
def ssim_loss(y_true, y_pred):
    return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2))


def PSNR(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, 2)

# Loss Function and Callbacks

In [ ]:
# early stopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", verbose=2, patience=10
)
# ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.8, patience=5, mode="min", verbose=1, min_lr=0.000001
)
# ModelCheckpoint
ch_point = tf.keras.callbacks.ModelCheckpoint(
    dir_cur+"\path\filename",
    monitor="val_loss",
    verbose=1,
    mode="min",
    save_best_only="True",
)

# CSVLogger
log_train = tf.keras.callbacks.CSVLogger(
   dir_cur+"\path\filename",
    separator=",",
    append=False,
)


# lr scheduler


def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.82
    epochs_drop = 5
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

lr_sch = tf.keras.callbacks.LearningRateScheduler(step_decay)

# HyperParameters

In [ ]:
# Autoencoder
Epochs = 150
BATCH_s = 16
initial_learning_rate = 0.0001

# Training

In [ ]:
autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)


autoencoder.fit(
    train_x,
    train_y,
    batch_size=BATCH_s,
    epochs=Epochs,
    validation_data=(valid_x, valid_y),
    validation_batch_size=BATCH_s,
    verbose=1,
    shuffle=False,
    callbacks=[early_stop, lr_sch, ch_point, log_train],
)

# Saving Loss and Accuracy

In [ ]:
Loss = autoencoder.history.history["loss"]
Val_Loss = autoencoder.history.history["val_loss"]
accuracy = autoencoder.history.history["accuracy"]
Val_accuracy = autoencoder.history.history["val_accuracy"]

np.save(
    dir_cur+"\path\filename", np.array(Loss)
)
np.save(
    dir_cur+"\path\filename",
    np.array(Val_Loss),
)
np.save(
   dir_cur+"\path\filename", np.array(accuracy)
)
np.save(
    dir_cur+"\path\filename",
    np.array(Val_accuracy),
)

# Loss Plot

In [ ]:
plt.figure(figsize=[8, 6])
plt.plot(Loss,c='#3300cc',linewidth=1.9,marker='o',label = 'Training loss (MSE)',linestyle = '-', markersize= 8, markerfacecolor = '#FFFFFF',mew=2,markevery=5)
plt.plot(Val_Loss,'#cc3300',linewidth=1.9,marker='X',label = 'Validation loss (MSE) ',linestyle = '--', markersize= 8, markerfacecolor = '#FFFFFF',mew=2,markevery=5)
plt.title("Model loss")
plt.legend(fontsize='large', frameon=False, ncol=1, loc='upper right')
plt.xlabel('Epoch', fontsize = 11)
plt.ylabel('MSE Loss' , fontsize = 11)
plt.grid(color="black", linestyle="-", linewidth=0.2)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.show()

# Accuracy Plot

In [ ]:
plt.figure(figsize=[8,6])
plt.plot(accuracy,c='#3300cc',linewidth=1.9,marker='o',label = 'Training Acc (SSIM)',linestyle = '-', markersize= 8, markerfacecolor = '#FFFFFF',mew=2,markevery=5)
plt.plot(Val_accuracy, '#cc3300',linewidth=1.9,marker='X',label = 'Validation Acc (SSIM) ',linestyle = '--', markersize= 8, markerfacecolor = '#FFFFFF',mew=2,markevery=5)
plt.title('Model accuracy')
plt.ylabel('SSIM')
plt.xlabel('Epoch')
plt.legend(fontsize='large', frameon=False, ncol=1, loc='best')
plt.xlabel('Epoch', fontsize = 11)
plt.ylabel('Accuracy' , fontsize = 11)
#plt.yticks(np.arange(0,1.01,0.04))
plt.ylim([0.8,1])
plt.grid(color = 'k', linestyle = '-', linewidth = 0.2)
plt.show()

# Loading The Saved Model

In [ ]:
autoencoder = tf.keras.models.load_model(
    dir_cur+"\path\filename", custom_objects = {"ssim_loss":ssim_loss,"PSNR":PSNR}
)

# Model Evaluation (Test Data)

In [ ]:
autoencoder.evaluate(test_x, test_y, verbose=1)

# Prediction

In [ ]:
predictions = np.argmax(autoencoder.predict(test_x),axis=3)
predictions = np.array(np.expand_dims(predictions,-1),dtype=np.float32)
np.save(r"path\filename",predictions)
print(predictions.shape)


# SSIM Metric

In [ ]:
ssim_metric = []
for i in range(len(test_x)):
    ssim_metric.append(ssim_loss(test_y[i], predictions[i]))
print(np.max(ssim_metric))
print(np.where(ssim_metric == np.max(ssim_metric)))
ssim_res = [np.min(ssim_metric), np.max(ssim_metric), np.mean(ssim_metric)]

file = open("filename.txt", "w+")
file.write(str(ssim_res))
file.close()

# PSNR Metric

In [ ]:
psnr_metric = []
for i in range(len(test_x)):
    psnr_metric.append(PSNR(test_y[i], predictions[i]))
print(np.max(psnr_metric))
print(np.where(psnr_metric == np.max(psnr_metric)))
psnr_res = [np.min(psnr_metric), np.max(psnr_metric), np.mean(psnr_metric)]

file_1 = open("filename.txt", "w+")
file_1.write(str(psnr_res))
file_1.close()

# Intersection Over Union (IOU)

In [ ]:
IoU_keras = tf.keras.metrics.MeanIoU (num_classes=3)
IoU_keras.update_state(test_y,predictions)
values = np.array(IoU_keras.get_weights()).reshape(3,3)
print(values)

print(f'Mean Iou =  {IoU_keras.result().numpy():.4f}')



class1_Iou = values[0,0]/(values[0,0]+values[0,1]+values[0,2]+values[1,0]+values[2,0])
class2_Iou = values[1,1]/(values[1,1]+values[1,0]+values[1,2]+values[0,1]+values[2,1])
class3_Iou = values[2,2]/(values[2,2]+values[2,0]+values[2,1]+values[0,2]+values[1,2])
print(f'IoU for class1 is: {class1_Iou:.4f}')
print(f'IoU for class2 is: {class2_Iou:.4f}')
print(f'IoU for class3 is: {class3_Iou:.4f}')

file_2 = open("filename.txt","w+")
file_2.write(f'Mean Iou =  {IoU_keras.result().numpy():.4f}')
file_2.write(f'\nIoU for class1 is: {class1_Iou:.4f}')
file_2.write(f'\nIoU for class2 is: {class2_Iou:.4f}')
file_2.write(f'\nIoU for class3 is: {class3_Iou:.4f}')
file_2.close()

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print(classification_report(np.ravel(test_y), np.ravel(predictions)))
rep = classification_report(np.ravel(test_y), np.ravel(predictions))
file_3 = open("filename.txt", "w+")
file_3.write(rep)
file_3.close()